<a href="https://colab.research.google.com/github/AndrewZou/Coursera_Capstone/blob/master/Segmenting_Clustering_Toronto_Downtown_Neighbourhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment of Segmenting and Clustering Neighborhoods in Toronto Part 3 Requirements

#### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

- Just make sure:
1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.




Import all required Python packages

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Upload the Toronto Neighbourhood GEO codes file

In [93]:
from google.colab import files
uploaded = files.upload()

Initialize the Pandas dataframe of Toronto Neighbourhood 

In [94]:
df_toronto_neighborhood = pd.read_csv(r"toronto_neighbourhood_geocodes.CSV")

df_toronto_neighborhood.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,latitude,longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.804930,-79.165837
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.754898,-79.197776
3,3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692


Drop an extra column 'Unnamed: 0'

In [95]:
df_toronto_neighborhood.drop(columns=['Unnamed: 0'], axis = 1, inplace=True)
df_toronto_neighborhood.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge, Malvern",43.804930,-79.165837
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.754898,-79.197776
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692


### Create a Toronto Neighbourhood map with markers

In [0]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_neighborhood['latitude'], df_toronto_neighborhood['longitude'], df_toronto_neighborhood['Borough'], df_toronto_neighborhood['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<img src="toronto_neighbourhood.png" alt="Toronto Neighbourhood" title="Toronto Neighbourhood Map" />

### Create a subset of Downtown area of city of Toronto

In [97]:
downtown_data = df_toronto_neighborhood[df_toronto_neighborhood['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M4W,Downtown Toronto,Rosedale,43.660483,-79.383602
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.655115,-79.380219
2,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655115,-79.380219
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.658153,-79.379080


#### Here is the Downtown map

In [0]:
address = 'Downtown, Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_data['latitude'], downtown_data['longitude'], downtown_data['Borough'], downtown_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_downtown)  
    
map_toronto_downtown

<img src="toronto_downtown.png" alt="Toronto Downtown" title="Toronto Downtown Map" />

## Utilizing the Foursquare API to explore the neighborhoods and segment them

#### Define Foursquare Credentials and Version¶

Now, let's get the top 100 venues that are in Downtown Toronto within a radius of 500 meters.

First, let's create the GET request URL. 

Send Request thru the API call.

Observing the result of the responsed JSON data.

In [99]:
# @hidden_cell
CLIENT_ID = 'URQCL0KJ4KGOL31CQW105ID4PCBVI0DCNXA3P12YF1GAVMKV' # your Foursquare ID
CLIENT_SECRET = 'JI4SDXR4RFB1JTHILGKJV2B1NXAIXXCPVWB22NRVU3NRXBMH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

latitude = downtown_data.loc[0, 'latitude'] # neighborhood latitude value
longitude = downtown_data.loc[0, 'longitude'] # neighborhood longitude value

#neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
#url

#send a request and observe the response
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5d1e6a8fd29cbb00236d3aa9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-51755dc7498ece19b7261641-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c5941735',
         'name': 'Sandwich Place',
         'pluralName': 'Sandwich Places',
         'primary': True,
         'shortName': 'Sandwiches'}],
       'id': '51755dc7498ece19b7261641',
       'location': {'address': '399 Yonge St.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Gerrard St.',
        'distance': 186,
        'formattedAddress': ['399 Yonge St. (Gerrard St.)',
         'Toronto ON',
         'Canada'],
        'labeledLatLngs': [{'label': '

Define function to parser the response JSON format data.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Initialize a Pandas DataFrame holding the JSON data

In [101]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Banh Mi Boys,Sandwich Place,43.659188,-79.382131
1,Hailed Coffee,Coffee Shop,43.658833,-79.383684
2,The Yoga Sanctuary,Yoga Studio,43.661499,-79.383636
3,Duke's Refresher + Bar,Bar,43.658980,-79.382949
4,Scaddabush Italian Kitchen & Bar,Italian Restaurant,43.658920,-79.382891


### Explore Neighborhoods in Downtown
Let's create a function to repeat the same process to all the neighborhoods in Downtown
¶


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Code to run the above function on each neighborhood and create a new dataframe called downtown_venues.

In [102]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['latitude'],
                                   longitudes=downtown_data['longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [103]:
downtown_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1694,Christie,43.649991,-79.385722,Mercatto,43.650243,-79.380820,Italian Restaurant
1695,Christie,43.649991,-79.385722,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
1696,Christie,43.649991,-79.385722,Five Guys,43.648953,-79.381481,Burger Joint
1697,Christie,43.649991,-79.385722,The Adelaide Hotel Toronto,43.649831,-79.380164,Hotel
1698,Christie,43.649991,-79.385722,FLOCK Rotisserie + Greens,43.650988,-79.381917,Salad Place


In [104]:
downtown_venues.shape

(1699, 7)

Let's check how many venues were returned for each neighborhood

In [105]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",100,100,100,100,100,100
Central Bay Street,75,75,75,75,75,75
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,45,45,45,45,45,45
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [106]:

print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 121 uniques categories.


### Analyze Each Neighborhood

In [107]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

downtown_onehot.shape

(1699, 121)

In [108]:
downtown_onehot.drop( columns = ['Neighborhood'], axis = 1, inplace = True)
downtown_onehot.head()

,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

downtown_onehot.head()

,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Rosedale
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale


In [110]:
# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])

downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [111]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Hobby Shop,Hookah Bar,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.030000,0.000000,0.00,0.000000,0.01,0.030000,0.01,0.010000,0.01,0.010000,0.00,0.02,0.020000,0.010000,0.010000,0.030000,0.020000,0.050000,0.060000,0.010000,0.00,0.010000,0.00,0.04,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.01,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.01,0.020000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.030000,0.000000,0.00,0.01,0.01,0.00,0.01,0.01,0.000000,0.01,0.00,0.010000,0.030000,0.010000,0.010000,0.01,0.010000,0.00,0.01,0.000000,0.00,0.01,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.02,0.01,0.000000,0.00,0.00,0.010000,0.00,0.03,0.000000,0.00,0.00,0.010000,0.000000,0.010000,0.01,0.01,0.00,0.010000,0.00,0.010000,0.00,0.010000,0.010000,0.000000,0.020000,0.00,0.00,0.030000,0.010000,0.02,0.01,0.01,0.00,0.00,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.030000,0.000000,0.00,0.000000,0.01,0.030000,0.01,0.010000,0.01,0.010000,0.00,0.02,0.020000,0.010000,0.010000,0.030000,0.020000,0.050000,0.060000,0.010000,0.00,0.010000,0.00,0.04,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.01,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.01,0.020000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.030000,0.000000,0.00,0.01,0.01,0.00,0.01,0.01,0.000000,0.01,0.00,0.010000,0.030000,0.010000,0.010000,0.01,0.010000,0.00,0.01,0.000000,0.00,0.01,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.02,0.01,0.000000,0.00,0.00,0.010000,0.00,0.03,0.000000,0.00,0.00,0.010000,0.000000,0.010000,0.01,0.01,0.00,0.010000,0.00,0.010000,0.00,0.010000,0.010000,0.000000,0.020000,0.00,0.00,0.030000,0.010000,0.02,0.01,0.01,0.00,0.00,0.010000,0.000000,0.01,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.030000,0.000000,0.00,0.000000,0.01,0.030000,0.01,0.010000,0.01,0.010000,0.00,0.02,0.020000,0.010000,0.010000,0.030000,0.020000,0.050000,0.060000,0.010000,0.00,0.010000,0.00,0.04,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.01,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.01,0.020000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.030000,0.000000,0.00,0.01,0.01,0.00,0.01,0.01,0.000000,0.01,0.00,0.010000,0.03

In [112]:
downtown_grouped.shape

(18, 121)

#### Let's print each neighborhood along with the top 5 most common venues

In [113]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.06
1  Clothing Store  0.05
2  Cosmetics Shop  0.04
3      Restaurant  0.03
4        Tea Room  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.06
1  Clothing Store  0.05
2  Cosmetics Shop  0.04
3      Restaurant  0.03
4        Tea Room  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
            venue  freq
0     Coffee Shop  0.06
1  Clothing Store  0.05
2  Cosmetics Shop  0.04
3      Restaurant  0.03
4        Tea Room  0.03


----Cabbagetown, St. James Town----
            venue  freq
0     Coffee Shop  0.06
1  Clothing Store  0.05
2  Cosmetics Shop  0.04
3      Restaurant  0.03
4        Tea Room  0.03


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.23
1      Sandwich Place  0.05
2  Italian Restaurant  0.04
3                Café  0.04
4        Burger Joint  0.04


----Chinatown, 

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [116]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
1,Berczy Park,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
3,"Cabbagetown, St. James Town",Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
4,Central Bay Street,Coffee Shop,Sandwich Place,Burger Joint,Italian Restaurant,Café,Sushi Restaurant,Bubble Tea Shop,Bakery,Tea Room,Seafood Restaurant


## Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [117]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 2, 4, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [118]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.660483,-79.383602,3,Coffee Shop,Japanese Restaurant,Sandwich Place,Italian Restaurant,Burger Joint,Clothing Store,Seafood Restaurant,Ramen Restaurant,Gastropub,Café
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.655115,-79.380219,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
2,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875,4,Coffee Shop,Sushi Restaurant,Park,Burger Joint,Japanese Restaurant,Italian Restaurant,Gastropub,Persian Restaurant,Nightclub,Mexican Restaurant
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655115,-79.380219,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.658153,-79.379080,3,Coffee Shop,Clothing Store,Café,Ramen Restaurant,Middle Eastern Restaurant,Burger Joint,Hotel,Restaurant,Sandwich Place,Lounge


## Finally, let's visualize the resulting clusters

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['latitude'], downtown_merged['longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<img src="toronto_downtown_clustering.png" alt="Toronto Downtown Venus Clustering" title="Toronto Downtown Map" />

## Examine Clusters

In [121]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
3,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
6,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
8,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
10,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
11,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
12,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery
13,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Café,Restaurant,Italian Restaurant,Tea Room,Middle Eastern Restaurant,Bakery


In [122]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Coffee Shop,Sandwich Place,Burger Joint,Italian Restaurant,Café,Sushi Restaurant,Bubble Tea Shop,Bakery,Tea Room,Seafood Restaurant


In [123]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Hotel,Restaurant,American Restaurant,Pizza Place,Salad Place


In [124]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Japanese Restaurant,Sandwich Place,Italian Restaurant,Burger Joint,Clothing Store,Seafood Restaurant,Ramen Restaurant,Gastropub,Café
4,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Ramen Restaurant,Middle Eastern Restaurant,Burger Joint,Hotel,Restaurant,Sandwich Place,Lounge


In [125]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Park,Burger Joint,Japanese Restaurant,Italian Restaurant,Gastropub,Persian Restaurant,Nightclub,Mexican Restaurant
